In [8]:
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer # Pour gérer les valeurs manquantes
    file_path = r'C:\Users\marjane\OneDrive\Bureau\COURSES\Projet_PFA\fraud_detection_app\data\bank_transactions_data_2.csv' # Chemin relatif
    df = pd.read_csv(file_path)
    print("Dataset chargé pour prétraitement.")
    df.head() 

Dataset chargé pour prétraitement.


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


In [9]:
import os

# --- Code pour changer le répertoire de travail ---
# Remplacez par le chemin ABSOLU correct de la racine de votre projet
project_root_path = 'C:\\Users\\marjane\\OneDrive\\Bureau\\COURSES\\Projet_PFA\\fraud_detection_app'  

# Changer le répertoire de travail
try:
    os.chdir(project_root_path)
    print(f"Répertoire de travail changé pour : {os.getcwd()}")
except FileNotFoundError:
    print(f"Erreur : Le chemin spécifié '{project_root_path}' n'a pas été trouvé. Vérifiez le chemin.")
except Exception as e:
    print(f"Une erreur est survenue lors du changement de répertoire : {e}")
# --- Fin du code pour changer le répertoire ---

Répertoire de travail changé pour : C:\Users\marjane\OneDrive\Bureau\COURSES\Projet_PFA\fraud_detection_app


In [10]:
df.isnull().sum()

TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64

In [11]:
# --- Étape 2 : Gérer les Outliers / Transformations Logarithmiques ---

# Identifier les colonnes numériques à transformer (basé sur votre EDA)
# Les plus courantes sont TransactionAmount et AccountBalance si elles sont asymétriques
cols_to_transform_log = ['TransactionAmount', 'AccountBalance']

for col in cols_to_transform_log:
    # Vérifier si la colonne existe et si elle est numérique
    if col in df.columns and np.issubdtype(df[col].dtype, np.number):
        # Appliquer la transformation log(1+x) pour gérer les valeurs 0 et les outliers
        # Le fillna(0) gère ici les éventuels NaNs restants avant la transformation log1p
        # Assurez-vous que votre gestion principale des NaNs à l'étape 1 est suffisante,
        # mais ce fillna(0) ici est une sécurité et gère aussi les valeurs 0 réelles.
        df[f'{col}_log'] = np.log1p(df[col].fillna(0))
    # Note: Pas de else print ici car vous êtes dans un notebook
    # Vous pouvez vérifier après en affichant df.head() ou df.columns


# N'oubliez pas d'utiliser ensuite les nouvelles colonnes '_log'
# à la place des colonnes originales dans la sélection des features finales (Étape 5)

In [12]:
#verification de nouveau colones ajouter TransactionAmount_log et AccountBalance_log
df.head()


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionAmount_log,AccountBalance_log
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08,2.714032,8.539583
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35,5.932882,9.529515
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04,4.846468,7.024071
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06,5.223055,9.056030
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39,2.670694,8.913335


In [13]:
# 1-checking  le format datetime pour les colonnes de date
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'], errors='coerce')

print("Colonnes de date converties au format datetime (les erreurs sont maintenant NaT)") 

Colonnes de date converties au format datetime (les erreurs sont maintenant NaT)


In [14]:
#Extraire l'heure (entre 0 et 23)
df['Hour'] = df['TransactionDate'].dt.hour

# Extraire le jour de la semaine (entre 0 pour Lundi et 6 pour Dimanche)
df['DayOfWeek'] = df['TransactionDate'].dt.dayofweek

In [15]:
#checking de creation des deux nouvelle colones: Hour et DayOfWeek, va permettre à K-means de regrouper les transactions qui se ressemblent non seulement par le montant, la localisation, etc., mais aussi par le moment où elles ont eu lieu. 
df.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionAmount_log,AccountBalance_log,Hour,DayOfWeek
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08,2.714032,8.539583,16,1
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35,5.932882,9.529515,16,1
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04,4.846468,7.024071,18,0
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06,5.223055,9.056030,16,4
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39,2.670694,8.913335,17,0


In [16]:
# Étape 3.1 : S'assurer que les dates sont datetime (déjà fait ci-dessus, mais bien de le confirmer)
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
# df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'], errors='coerce') # Si vous utilisez cette colonne aussi pour d'autres calculs

# Supprimer les lignes où la date de transaction est invalide si nécessaire (ou gérer dans l'étape 1)
df.dropna(subset=['TransactionDate'], inplace=True)


# Étape 3.2 : Trier les données par AccountID puis par TransactionDate
# Important pour que .diff() calcule l'écart avec la transaction précédente DU MÊME compte
# On travaille directement sur df maintenant car on est dans le notebook de prétraitement
df = df.sort_values(by=['AccountID', 'TransactionDate'])


# Étape 3.3 : Calculer la différence de temps (Timedelta) par groupe AccountID
# .diff() calcule la différence entre la ligne actuelle et la ligne précédente *dans chaque groupe*
df['TimeDelta'] = df.groupby('AccountID')['TransactionDate'].diff()


# Étape 3.4 : Convertir la différence de temps en nombre total de secondes
df['TimeSinceLastTransaction_s'] = df['TimeDelta'].dt.total_seconds()


# Étape 3.5 : Gérer les valeurs manquantes (NaN) dans TimeSinceLastTransaction_s
# Les premières transactions de chaque compte auront NaN ici.
# Imputer avec la médiane est une approche courante et moins sensible aux extrêmes que la moyenne.
median_time_diff = df['TimeSinceLastTransaction_s'].median()
df['TimeSinceLastTransaction_s'] = df['TimeSinceLastTransaction_s'].fillna(median_time_diff)
# Optionnel : Supprimer la colonne intermédiaire 'TimeDelta' si vous n'en avez plus besoin
df = df.drop('TimeDelta', axis=1)

# Pour voir le résultat dans Jupyter, mettez la dernière ligne que vous voulez afficher
df[['AccountID', 'TransactionDate', 'TimeSinceLastTransaction_s', 'Hour', 'DayOfWeek']].head()

,AccountID,TransactionDate,TimeSinceLastTransaction_s,Hour,DayOfWeek
1312,AC00001,2023-09-15 17:00:20,3370377.0,17,4
2016,AC00001,2023-11-14 16:56:34,5183774.0,16,1
2120,AC00002,2023-01-10 16:00:32,3370377.0,16,1
20,AC00002,2023-02-28 16:36:58,4235786.0,16,1
1476,AC00002,2023-05-05 16:35:44,5702326.0,16,4


In [17]:
# identification des colonnes a encoder :
# Colonnes catégorielles à encoder (basé sur votre EDA, exclure Location)
categorical_features = ['TransactionType', 'Channel', 'CustomerOccupation']

# Affichez la liste pour confirmation (dernière ligne de la cellule)
categorical_features

['TransactionType', 'Channel', 'CustomerOccupation']

In [18]:
    # Initialiser l'encodeur One-Hot
    # handle_unknown='ignore': gère les catégories qui pourraient apparaître dans les nouvelles données
    # sparse_output=False: retourne un tableau NumPy dense (plus facile à utiliser avec Pandas et Sklearn)
    from sklearn.preprocessing import OneHotEncoder # Assurez-vous que OneHotEncoder est importé au début

    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    # Affichez l'encodeur pour confirmation
    encoder

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [19]:
    # Appliquer l'encodage One-Hot aux colonnes sélectionnées
    # Assurez-vous de sélectionner uniquement les colonnes nécessaires de votre DataFrame 'df'
    encoded_features = encoder.fit_transform(df[categorical_features])

    # Affichez la forme du tableau résultant
    encoded_features.shape

(2512, 9)

In [20]:
    # Créer un DataFrame à partir des features encodées
    # get_feature_names_out() donne les noms comme 'Channel_Online'
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

    # Affichez les premières lignes du nouveau DataFrame encodé
    encoded_df.head()

,TransactionType_Credit,TransactionType_Debit,Channel_ATM,Channel_Branch,Channel_Online,CustomerOccupation_Doctor,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [21]:
    # Joindre le DataFrame encodé au DataFrame principal
    # Concaténation par colonnes (axis=1)
    # Assurez-vous que les index correspondent bien!
    # Si vous avez reset_index dans une étape précédente, c'est bon.
    df_processed = pd.concat([df, encoded_df], axis=1)

    # Affichez les premières lignes du DataFrame combiné pour voir les nouvelles colonnes
    df_processed.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,...,TimeSinceLastTransaction_s,TransactionType_Credit,TransactionType_Debit,Channel_ATM,Channel_Branch,Channel_Online,CustomerOccupation_Doctor,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student
1312,TX001313,AC00001,47.79,2023-09-15 17:00:20,Debit,Denver,D000649,59.12.96.11,M034,Branch,...,3370377.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2016,TX002017,AC00001,212.97,2023-11-14 16:56:34,Debit,Atlanta,D000492,45.241.13.208,M003,Online,...,5183774.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2120,TX002121,AC00002,476.99,2023-01-10 16:00:32,Debit,San Diego,D000594,113.137.153.101,M022,Online,...,3370377.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
20,TX000021,AC00002,59.32,2023-02-28 16:36:58,Debit,Los Angeles,D000152,116.44.12.250,M040,Branch,...,4235786.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1476,TX001477,AC00002,12.62,2023-05-05 16:35:44,Debit,El Paso,D000475,93.160.83.196,M068,Branch,...,5702326.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
    # Supprimer les colonnes catégorielles originales qui ont été encodées
    df_processed = df_processed.drop(categorical_features, axis=1)

    # Affichez les premières lignes et la liste des colonnes pour vérifier qu'elles sont parties
    df_processed.head()
    # Ou pour juste voir les colonnes :
    # df_processed.columns.tolist()

,TransactionID,AccountID,TransactionAmount,TransactionDate,Location,DeviceID,IP Address,MerchantID,CustomerAge,TransactionDuration,...,TimeSinceLastTransaction_s,TransactionType_Credit,TransactionType_Debit,Channel_ATM,Channel_Branch,Channel_Online,CustomerOccupation_Doctor,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student
1312,TX001313,AC00001,47.79,2023-09-15 17:00:20,Denver,D000649,59.12.96.11,M034,25,37,...,3370377.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2016,TX002017,AC00001,212.97,2023-11-14 16:56:34,Atlanta,D000492,45.241.13.208,M003,59,178,...,5183774.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2120,TX002121,AC00002,476.99,2023-01-10 16:00:32,San Diego,D000594,113.137.153.101,M022,23,187,...,3370377.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
20,TX000021,AC00002,59.32,2023-02-28 16:36:58,Los Angeles,D000152,116.44.12.250,M040,71,38,...,4235786.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1476,TX001477,AC00002,12.62,2023-05-05 16:35:44,El Paso,D000475,93.160.83.196,M068,33,61,...,5702326.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:


# Étape 5 : Sélection des Features Finale 

# Liste des colonnes que nous voulons garder pour le modèle (basé sur votre EDA et prétraitement)
# Inclure les numériques originales pertinentes, les dérivées, et les encodées
columns_to_keep = [
    'TransactionAmount', # Ou 'TransactionAmount_log' si vous l'avez créée et préférez l'utiliser
    'CustomerAge',
    'TransactionDuration',
    'AccountBalance', # Ou 'AccountBalance_log'
    'LoginAttempts',
    'Hour',
    'DayOfWeek',
    'TimeSinceLastTransaction_s',
    # Ajouter ici TOUS les noms des nouvelles colonnes créées par le One-Hot Encoding
    # Vous pouvez obtenir cette liste à partir de votre DataFrame 'encoded_df' de l'étape 4,
    # ou directement depuis df_processed si vous l'avez déjà fusionné et supprimé les originales.
    # Par exemple, si vous aviez la liste des features catégorielles:
    # 'TransactionType_Credit', 'TransactionType_Debit',
    # 'Channel_ATM', 'Channel_Branch', 'Channel_Online',
    # 'CustomerOccupation_Doctor', 'CustomerOccupation_Engineer', 'CustomerOccupation_Retired', 'CustomerOccupation_Student',
    # ... etc. Assurez-vous d'utiliser les noms exacts présents dans df_processed.
]

# Une façon plus robuste d'obtenir les noms des colonnes encodées si vous avez conservé 'encoder' de l'étape 4:
# encoded_column_names = list(encoder.get_feature_names_out(categorical_features))
# columns_to_keep.extend(encoded_column_names)
# columns_to_keep = list(set(columns_to_keep)) # Supprimer les doublons au cas où

# Alternative simple: identifier les colonnes à DROPER et garder le reste
columns_to_drop = [
    'TransactionID',
    'AccountID', # Identifiant, pas un feature pour K-means
    'TransactionDate', # Date brute, remplacée par features dérivées
    'PreviousTransactionDate', # Date brute
    'Location', # Haute cardinalité, non encodée de manière pertinente
    'DeviceID', # Identifiant
    'IP Address', # Identifiant
    'MerchantID', # Identifiant
    # Les colonnes catégorielles originales ('TransactionType', 'Channel', 'CustomerOccupation')
    # devraient déjà avoir été supprimées à la fin de l'étape 4. Double-vérifiez si nécessaire.
    # Par exemple: 'TransactionType', 'Channel', 'CustomerOccupation'
    # Si vous avez gardé des colonnes intermédiaires comme 'TimeDelta', supprimez-les aussi.
    'TimeDelta' # Si cette colonne existe encore
]

# Assurez-vous de ne pas essayer de dropper des colonnes qui n'existent pas
columns_to_drop_existing = [col for col in columns_to_drop if col in df_processed.columns]

# Créer la matrice de features X en supprimant les colonnes inutiles
X = df_processed.drop(columns_to_drop_existing, axis=1)


In [24]:
#creation  d'un nouveau tableau de données (X) qui contient uniquement les informations pertinentes et numériques, prêtes pour la mise à l'échelle avant de les donner au modèle. 
X.head()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmount_log,AccountBalance_log,Hour,DayOfWeek,TimeSinceLastTransaction_s,TransactionType_Credit,TransactionType_Debit,Channel_ATM,Channel_Branch,Channel_Online,CustomerOccupation_Doctor,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student
1312,47.79,25,37,1,1649.92,3.887525,7.409088,17,4,3370377.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2016,212.97,59,178,1,4180.40,5.365836,8.338401,16,1,5183774.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2120,476.99,23,187,1,1154.48,6.169590,7.052271,16,1,3370377.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
20,59.32,71,38,1,5750.89,4.099664,8.657284,16,1,4235786.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1476,12.62,33,61,1,6420.47,2.611539,8.767402,16,4,5702326.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
  # Étape 6 : Normaliser / Standardiser les Features

 #  X est le  DataFrame final prêt pour le scaling

from sklearn.preprocessing import StandardScaler # Importer le scaler

    # Initialiser le StandardScaler
scaler = StandardScaler()

    # Afficher le scaler pour confirmation
    # scaler
import joblib
joblib.dump(scaler, 'models/scaler.pkl')
print("Scaler saved to models/scaler.pkl")

Scaler saved to models/scaler.pkl


In [26]:
    # Appliquer la standardisation à l'ensemble de la matrice de features X
    # Le résultat est un tableau NumPy
X_scaled = scaler.fit_transform(X)

    # Affichez les premières lignes du tableau standardisé pour voir le résultat
X_scaled[:5] # Affiche les 5 premières lignes du tableau NumPy



array([[-0.85582028, -1.10598375, -1.18146552, -0.20679365, -0.88826553,
        -1.03125   , -0.62885835,  0.51142281,  1.68877267, -0.26720755,
        -0.54053776,  0.54053776,  1.41972048, -0.72662269, -0.69049139,
        -0.57918897, -0.57551157, -0.55957194,  1.68030946],
       [-0.28991853,  0.80534666,  0.8342649 , -0.20679365, -0.23945211,
         0.18165364,  0.21792873, -0.85672852, -0.29665492,  0.18081182,
         1.85000952, -1.85000952, -0.704364  , -0.72662269,  1.44824397,
        -0.57918897, -0.57551157, -0.55957194,  1.68030946],
       [ 0.6146061 , -1.21841495,  0.96292855, -0.20679365, -1.01529602,
         0.84110656, -0.95398861, -0.85672852, -0.29665492, -0.26720755,
        -0.54053776,  0.54053776, -0.704364  , -0.72662269,  1.44824397,
        -0.57918897, -0.57551157, -0.55957194,  1.68030946],
       [-0.81631884,  1.47993387, -1.16716956, -0.20679365,  0.1632205 ,
        -0.85719767,  0.50849323, -0.85672852, -0.29665492, -0.05339888,
        -0.540

In [27]:
    # Convertir le tableau NumPy standardisé en DataFrame Pandas
    # Utiliser les noms de colonnes du DataFrame original X
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    # Affichez les premières lignes et les statistiques descriptives du DataFrame standardisé
    # Vérifiez que les moyennes sont proches de 0 et les écarts-types proches de 1
    X_scaled_df.head()
    # X_scaled_df.describe()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmount_log,AccountBalance_log,Hour,DayOfWeek,TimeSinceLastTransaction_s,TransactionType_Credit,TransactionType_Debit,Channel_ATM,Channel_Branch,Channel_Online,CustomerOccupation_Doctor,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student
0,-0.855820,-1.105984,-1.181466,-0.206794,-0.888266,-1.031250,-0.628858,0.511423,1.688773,-0.267208,-0.540538,0.540538,1.419720,-0.726623,-0.690491,-0.579189,-0.575512,-0.559572,1.680309
1,-0.289919,0.805347,0.834265,-0.206794,-0.239452,0.181654,0.217929,-0.856729,-0.296655,0.180812,1.850010,-1.850010,-0.704364,-0.726623,1.448244,-0.579189,-0.575512,-0.559572,1.680309
2,0.614606,-1.218415,0.962929,-0.206794,-1.015296,0.841107,-0.953989,-0.856729,-0.296655,-0.267208,-0.540538,0.540538,-0.704364,-0.726623,1.448244,-0.579189,-0.575512,-0.559572,1.680309
3,-0.816319,1.479934,-1.167170,-0.206794,0.163220,-0.857198,0.508493,-0.856729,-0.296655,-0.053399,-0.540538,0.540538,1.419720,-0.726623,-0.690491,-0.579189,-0.575512,-0.559572,1.680309
4,-0.976312,-0.656259,-0.838362,-0.206794,0.334900,-2.078153,0.608833,-0.856729,1.688773,0.308926,-0.540538,0.540538,1.419720,-0.726623,-0.690491,-0.579189,-0.575512,-0.559572,1.680309


In [28]:
import os
print(os.getcwd())

C:\Users\marjane\OneDrive\Bureau\COURSES\Projet_PFA\fraud_detection_app


In [29]:

# Étape 7 : Sauvegarder le Dataset Prétraité

# Assurez-vous que X_scaled_df est le DataFrame final standardisé
# Assurez-vous que X est le DataFrame avant standardisation (créé juste avant le scaling)

# Définir les chemins où sauvegarder les fichiers
output_scaled_file_path = 'data/bank_transactions_processed_scaled.csv' # Nom plus précis
output_raw_features_file_path = 'data/bank_transactions_features_raw.csv' # Nom pour X avant scaling

# Sauvegarder le DataFrame standardisé
X_scaled_df.to_csv(output_scaled_file_path, index=False)

# --- AJOUTEZ CETTE LIGNE POUR SAUVEGARDER X ---
# Sauvegarder le DataFrame X (avant standardisation)
X.to_csv(output_raw_features_file_path, index=False)
# ------------------------------------------

# Afficher un message de confirmation
f"Datasets prétraités sauvegardés avec succès :\n - Scaled Features sous {output_scaled_file_path}\n - Raw Features sous {output_raw_features_file_path}"

'Datasets prétraités sauvegardés avec succès :\n - Scaled Features sous data/bank_transactions_processed_scaled.csv\n - Raw Features sous data/bank_transactions_features_raw.csv'